In [92]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets

pn.extension()

%matplotlib inline

In [93]:
# Read the Mapbox API key
mapbox_token = os.getenv("MAPBOX_API_KEY")
load_dotenv()

True

In [94]:
# Read the ATLAS data into a Pandas DataFrame
file_path = Path("Resources/Raw/ATLAS.csv")
ATLAS = pd.read_csv(file_path, index_col="Customers")
ATLAS.head()

,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Dates of service
Customers,,,,,,
Louisiana Board of Regents,3/20/2015,ATLAS 315,"$72,000.00",1 Year,4700-0-00-00000-18-0000,1 Year Subscription 3/18/15 to 6/30/16
Emporia State University,5/28/2015,AJ501,"$3,500.00",1 Year,4700-0-00-00000-16-0000,1 Year Subscription 6/1/15 to 6/30/16
Morehouse State University,6/23/2015,AJ502,"$3,500.00",1 Year,4700-0-00-00000-17-0000,1 Year Subscription 6/10/15 to 6/30/16
State University of NY Postdam,6/26/2015,AJ503,"$6,500.00",1 Year,4700-0-00-00000-32-0000,1 Year Subscription 6/1/15 to 6/30/16
ST. Mary's College of Maryland,10/7/2015,AJ504,$750.00,1 Year,4700-0-00-00000-20-0000,1 Year Subscription 10/05/15 to 9/30/16


In [100]:
# Cleanup/preprocessing

## Cean up dates column (TODO functionalize)
dates_of_service = ATLAS.iloc[0:6]["Dates of service "]

start_date_list = []
end_date_list = []

for date_of_service_str in dates_of_service:
    split1 = date_of_service_str.find("Subscription")
    split2 = split1 + len("Subscription")
    date_range = date_of_service_str[split2:].strip()

    split1 = date_range.find("to")
    split2 = split1 + len("to")
    date1_str = date_range[0:split1].strip()
    date2_str = date_range[split2:].strip()

    print(f"date1_str: {date1_str}  date2_str: {date2_str}")
    date1 = pd.Timestamp(date1_str, tz="America/New_York").isoformat()
    date2 = pd.Timestamp(date2_str, tz="America/New_York").isoformat()
    print(f"date1_str: {date1_str}  date1: {date1}")
    print(f"date2_str: {date2_str}  date2: {date2}")

    start_date_list.append(date1)
    end_date_list.append(date2)

ATLAS_with_fixed_dates = pd.concat([ATLAS, start_date_list, end_date_list])



date1_str: 3/18/15  date2_str: 6/30/16
date1_str: 3/18/15  date1: 2015-03-18T00:00:00-04:00
date2_str: 6/30/16  date2: 2016-06-30T00:00:00-04:00
date1_str: 6/1/15  date2_str: 6/30/16
date1_str: 6/1/15  date1: 2015-06-01T00:00:00-04:00
date2_str: 6/30/16  date2: 2016-06-30T00:00:00-04:00
date1_str: 6/10/15  date2_str: 6/30/16
date1_str: 6/10/15  date1: 2015-06-10T00:00:00-04:00
date2_str: 6/30/16  date2: 2016-06-30T00:00:00-04:00
date1_str: 6/1/15  date2_str: 6/30/16
date1_str: 6/1/15  date1: 2015-06-01T00:00:00-04:00
date2_str: 6/30/16  date2: 2016-06-30T00:00:00-04:00
date1_str: 10/05/15  date2_str: 9/30/16
date1_str: 10/05/15  date1: 2015-10-05T00:00:00-04:00
date2_str: 9/30/16  date2: 2016-09-30T00:00:00-04:00
date1_str: 10/01/15  date2_str: 6/30/17
date1_str: 10/01/15  date1: 2015-10-01T00:00:00-04:00
date2_str: 6/30/17  date2: 2017-06-30T00:00:00-04:00


In [96]:
## Extract paying customers
#TODO Clean up customer names
paying_customers = ATLAS.reset_index()["Customers"].sort_values().drop_duplicates()
paying_customers.head()

129    Alabama A&M University-College of Education
146                        Albertus Magnus College
178                     Albuquerque Public Schools
114                       Alcorn State University 
635                  Alderson Broaddus University 
Name: Customers, dtype: object